# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

In [21]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
from pprint import pprint
import json
import time

# Import API key
from config import g_key

### Store Part I results into DataFrame

In [22]:
cities_df = pd.read_csv("weather.csv")
cities_df

,City,Country,Latitude,Longitude,Max Temp (f),Humidity (%),Cloud Cover (%),Wind Speed (mph)
0,Castro,BR,-24.7911,-50.0119,55.63,92,99,5.73
1,Ushuaia,AR,-54.8000,-68.3000,37.06,93,75,25.32
2,Kutum,SD,14.2000,24.6667,68.00,20,0,5.91
3,East London,ZA,-33.0153,27.9116,68.94,77,0,2.30
4,Castelo do Piauí,BR,-5.3222,-41.5525,72.63,91,34,2.28
...,...,...,...,...,...,...,...,...
551,Madurai,IN,9.9333,78.1167,87.82,66,40,3.44
552,La Grande,US,45.3246,-118.0877,57.06,69,96,8.81
553,Nouadhibou,MR,20.9310,-17.0347,69.78,68,97,6.91
554,Oistins,BB,13.0667,-59.5333,78.89,78,40,21.85


### Humidity Heatmap

In [23]:
#configure gmaps
import gmaps
gmaps.configure(api_key = g_key)

In [24]:
 # Store latitude and longitude in locations
locations = cities_df[["Latitude", "Longitude"]]

# Fill NaN values and convert to float
humidity = cities_df["Humidity (%)"]

# Plot Heatmap
fig = gmaps.Map()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity)

# Add layer
fig.add_layer(heat_layer)

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [25]:
 cities_df = cities_df.loc[cities_df['Max Temp (f)'] < 80, :]
 cities_df = cities_df.loc[cities_df['Max Temp (f)'] > 70, :]
 cities_df = cities_df.loc[cities_df['Wind Speed (mph)'] < 10, :]
 cities_df = cities_df.loc[cities_df['Cloud Cover (%)'] == 0, :]
 cities_df.head()

,City,Country,Latitude,Longitude,Max Temp (f),Humidity (%),Cloud Cover (%),Wind Speed (mph)
27,Makkah al Mukarramah,SA,21.4267,39.8261,79.02,61,0,0.69
125,Tura,IN,25.5198,90.2201,76.15,48,0,2.86
145,Kibala,TD,9.1104,18.3463,73.35,94,0,0.00
146,Sydney,AU,-33.8679,151.2073,72.34,37,0,3.00
192,Nādbai,IN,27.2333,77.2000,77.40,26,0,0.65


In [26]:
# Customize DataFrame by requested conditions.
weather_df = cities_df.loc[(cities_df["Max Temp (f)"]>=70) & (cities_df["Max Temp (f)"]<=80) &
                   (cities_df["Wind Speed (mph)"]<=10) & (cities_df["Cloud Cover (%)"]==0)].dropna()
weather_df

,City,Country,Latitude,Longitude,Max Temp (f),Humidity (%),Cloud Cover (%),Wind Speed (mph)
27,Makkah al Mukarramah,SA,21.4267,39.8261,79.02,61,0,0.69
125,Tura,IN,25.5198,90.2201,76.15,48,0,2.86
145,Kibala,TD,9.1104,18.3463,73.35,94,0,0.00
146,Sydney,AU,-33.8679,151.2073,72.34,37,0,3.00
192,Nādbai,IN,27.2333,77.2000,77.40,26,0,0.65
346,Doiwāla,IN,30.1833,78.1167,72.32,42,0,4.41
351,Morón,AR,-34.6534,-58.6198,71.35,75,0,3.96
365,Port Macquarie,AU,-31.4333,152.9167,78.76,35,0,3.27
450,San Patricio,US,28.0170,-97.5169,73.24,78,0,7.61
453,Tehri,IN,30.3833,78.4833,70.34,39,0,4.18


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [27]:
hotel_df = weather_df
hotel_df['Hotel Name'] = np.nan

target_search = "Hotel"
target_radius = 5000
target_type = "lodging"

idx = 0
for index, row in hotel_df.iterrows():
    # set up a parameters dictionary
    params = {
        "location": f"{row['Latitude']}, {row['Longitude']}",
        "keyword": target_search,
        "radius": target_radius,
        "type": target_type,
        "key": g_key
    }

    # base url
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # run a request using our params dictionary
    response = requests.get(base_url, params=params)
    response_json = response.json()
    results = response_json['results']
    if len(results) > 0:
        name = response_json['results'][0]['name']
        hotel_df.iloc[idx, -1] = name
    idx = idx + 1


hotel_df

,City,Country,Latitude,Longitude,Max Temp (f),Humidity (%),Cloud Cover (%),Wind Speed (mph),Hotel Name
27,Makkah al Mukarramah,SA,21.4267,39.8261,79.02,61,0,0.69,NaN
125,Tura,IN,25.5198,90.2201,76.15,48,0,2.86,NaN
145,Kibala,TD,9.1104,18.3463,73.35,94,0,0.00,NaN
146,Sydney,AU,-33.8679,151.2073,72.34,37,0,3.00,NaN
192,Nādbai,IN,27.2333,77.2000,77.40,26,0,0.65,NaN
346,Doiwāla,IN,30.1833,78.1167,72.32,42,0,4.41,NaN
351,Morón,AR,-34.6534,-58.6198,71.35,75,0,3.96,NaN
365,Port Macquarie,AU,-31.4333,152.9167,78.76,35,0,3.27,NaN
450,San Patricio,US,28.0170,-97.5169,73.24,78,0,7.61,NaN
453,Tehri,IN,30.3833,78.4833,70.34,39,0,4.18,NaN


In [28]:
#Add locations to list and convert humidity to float for weighting
hotel_name = hotel_df[["Hotel Name"]].astype(float)

In [29]:
# Store into variable named hotel_df
hotel_df = weather_df[["City","Country","Latitude","Longitude"]]

# Add a new column as "Hotel Name" 
hotel_df['Hotel Name'] = ""
hotel_df

/var/folders/nc/19j62rj52xdfh6h_lnm7zxy40000gn/T/ipykernel_34520/830790625.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df['Hotel Name'] = ""


,City,Country,Latitude,Longitude,Hotel Name
27,Makkah al Mukarramah,SA,21.4267,39.8261,
125,Tura,IN,25.5198,90.2201,
145,Kibala,TD,9.1104,18.3463,
146,Sydney,AU,-33.8679,151.2073,
192,Nādbai,IN,27.2333,77.2000,
346,Doiwāla,IN,30.1833,78.1167,
351,Morón,AR,-34.6534,-58.6198,
365,Port Macquarie,AU,-31.4333,152.9167,
450,San Patricio,US,28.0170,-97.5169,
453,Tehri,IN,30.3833,78.4833,


In [30]:
 # NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in narrowed_city_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

NameError: name 'narrowed_city_df' is not defined

In [ ]:
 # Add marker layer ontop of heat map
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
fig